<a href="https://colab.research.google.com/github/oveay/AM/blob/main/LAB_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Ведерников Ярослав, Соров Айаал, Хауров Никита. Группа М32011. Лабораторная работа №4

###1. Реализовать метод Гаусса с выбором ведущего элемента для решения СЛАУ

In [ ]:
import numpy as np

def gaussian_elimination(matrix, vector):
    n = len(matrix)

    # Прямой ход
    for i in range(n):
        # Выбор ведущего элемента
        max_index = i
        for j in range(i+1, n):
            if abs(matrix[j][i]) > abs(matrix[max_index][i]):
                max_index = j
        matrix[[i, max_index]] = matrix[[max_index, i]]
        vector[[i, max_index]] = vector[[max_index, i]]

        # Приведение матрицы к треугольному виду
        for j in range(i+1, n):
            factor = matrix[j][i] / matrix[i][i]
            matrix[j] -= factor * matrix[i]
            vector[j] -= factor * vector[i]

    # Обратный ход
    solution = np.zeros(n)
    for i in range(n-1, -1, -1):
        solution[i] = (vector[i] - np.dot(matrix[i][i+1:], solution[i+1:])) / matrix[i][i]

    return solution

# Пример использования
matrix = np.array([[2, 3, -1],
                   [4, 4, -3],
                   [2, -3, 1]], dtype=float)

vector = np.array([5, 3, -1], dtype=float)

result = gaussian_elimination(matrix, vector)
print(result)


[1. 2. 3.]


###2. Реализовать алгоритм LU-разложения с использованием разреженно-строчного (разреженно-столбцового) формата хранения матрицы, а также метод решения СЛАУ с использованием LU-разложения.

In [ ]:
import numpy as np
from scipy.sparse import lil_matrix, csr_matrix
from scipy.sparse.linalg import spsolve

def lu_decomposition_sparse(A):
    n = A.shape[0]
    L = lil_matrix((n, n), dtype=float)
    U = lil_matrix((n, n), dtype=float)

    for k in range(n):
        L[k, k] = 1.0
        U[k, k] = A[k, k] - L[k, :k].dot(U[:k, k].toarray().flatten())

        for j in range(k+1, n):
            U[k, j] = A[k, j] - L[k, :k].dot(U[:k, j].toarray().flatten())

        for i in range(k+1, n):
            L[i, k] = (A[i, k] - L[i, :k].dot(U[:k, k].toarray().flatten())) / U[k, k]

    return csr_matrix(L), csr_matrix(U)

# Пример использования
A = np.array([[2, 3, -1],
            [4, 4, -3],
            [2, -3, 1]])

# Создание разреженной матрицы в формате LIL
A_sparse = lil_matrix(A)

L, U = lu_decomposition_sparse(A_sparse)

print("L:")
print(L.toarray())
print("U:")
print(U.toarray())

# Решение СЛАУ с использованием LU-разложения
b = np.array([1, 0, -1])

# Решение СЛАУ
y = spsolve(L, b)
x = spsolve(U, y)

print("Solution:")
print(x)


L:
[[1. 0. 0.]
 [2. 1. 0.]
 [1. 3. 1.]]
U:
[[ 2.  3. -1.]
 [ 0. -2. -1.]
 [ 0.  0.  5.]]
Solution:
[0.  0.6 0.8]


###3. Реализовать итерационный метод решения СЛАУ (метол Зейделя, Якоби или верхней релаксации на выбор).

In [ ]:
import numpy as np

def jacobi_method(A, b, x0, max_iterations=100, tolerance=1e-6):
    n = len(A)
    x = np.copy(x0)
    x_new = np.zeros_like(x)
    residual = np.zeros(max_iterations)

    for iteration in range(max_iterations):
        for i in range(n):
            sum_term = 0.0
            for j in range(n):
                if j != i:
                    sum_term += A[i, j] * x[j]
            x_new[i] = (b[i] - sum_term) / A[i, i]

        residual[iteration] = np.linalg.norm(b - np.dot(A, x_new))

        if np.linalg.norm(x - x_new) < tolerance:
            break

        x = np.copy(x_new)

    return x, iteration + 1, residual[:iteration+1]

# Пример использования
A = np.array([[4, -1, 0], [-1, 4, -1], [0, -1, 4]])
b = np.array([1, 4, 2])
x0 = np.zeros_like(b)

solution, iterations, residual = jacobi_method(A, b, x0)
print("Решение:", solution)
print("Количество итераций:", iterations)
print("Остатки:", residual)


Решение: [0 1 0]
Количество итераций: 2
Остатки: [3.60555128 3.60555128]


###4. Провести исследование реализованных методов на системах с матрицами A^(k) число обусловленности которых регулируется за счет изменения диагонального преобладания. Внедиагональные элементы матрицы A^(k) выбираются случайным образом из множества

In [ ]:
import numpy as np

def gaussian_elimination(matrix, vector):
    n = len(matrix)

    for i in range(n):
        max_index = i
        for j in range(i+1, n):
            if abs(matrix[j][i]) > abs(matrix[max_index][i]):
                max_index = j
        matrix[[i, max_index]] = matrix[[max_index, i]]
        vector[[i, max_index]] = vector[[max_index, i]]

        for j in range(i+1, n):
            factor = matrix[j][i] / matrix[i][i]
            matrix[j] -= factor * matrix[i]
            vector[j] -= factor * vector[i]

    solution = np.zeros(n)
    for i in range(n-1, -1, -1):
        solution[i] = (vector[i] - np.dot(matrix[i][i+1:], solution[i+1:])) / matrix[i][i]

    return solution

def lu_decomposition(A):
    n = A.shape[0]
    L = np.eye(n)
    U = np.zeros((n, n))

    if np.linalg.det(A) == 0:
        raise ValueError('Матрица является вырожденной')

    for k in range(n):
        U[k, k:] = A[k, k:] - L[k, :k].dot(U[:k, k:])
        L[k+1:, k] = (A[k+1:, k] - L[k+1:, :k].dot(U[:k, k])) / U[k, k]

    return L, U

def jacobi_method(A, b, x0, max_iterations=100, tolerance=1e-6):
    n = len(A)
    x = np.copy(x0)
    x_new = np.zeros_like(x)
    residual = np.zeros(max_iterations)

    for iteration in range(max_iterations):
        for i in range(n):
            sum_term = 0.0
            for j in range(n):
                if j != i:
                    sum_term += A[i, j] * x[j]
            x_new[i] = (b[i] - sum_term) / A[i, i]

        residual[iteration] = np.linalg.norm(b - np.dot(A, x_new))

        if np.linalg.norm(x - x_new) < tolerance:
            break

        x = np.copy(x_new)

    return x, iteration + 1, residual[:iteration+1]

def solve_linear_system(A, F, method):
    if method == 'Gaussian':
        return gaussian_elimination(A, F)
    elif method == 'LU':
        try:
            L, U = lu_decomposition(A)
            y = np.linalg.solve(L, F)
            x = np.linalg.solve(U, y)
            return x
        except np.linalg.LinAlgError:
            return gaussian_elimination(A, F)
    elif method == 'Jacobi':
        x0 = np.zeros_like(F)
        return jacobi_method(A, F, x0)[0]
    else:
        raise ValueError('Неподдерживаемый метод')

# Создание матрицы A^k
def create_matrix(k, n):
    A = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if i == 0:
                A[i, j] = -1 * A[i+1, j] + 10**(-k)
            else:
                A[i, j] = np.random.choice([-4, -3, -2, -1, 0])

    return A

# Создание вектора F^k
def create_vector(n):
    return np.arange(1, n+1)

# Параметры исследования
k_values = [1, 2, 3]
n = 3

# Исследование для каждого значения k
for k in k_values:
    print(f"Исследование для k={k}")
    A = create_matrix(k, n)
    F = create_vector(n)

    # Точное решение
    exact_solution = np.linalg.solve(A, F)
    print("Точное решение:")
    print(exact_solution)

    # Решение с использованием метода Гаусса с выбором ведущего элемента
    gaussian_solution = solve_linear_system(A, F, 'Gaussian')
    print("Решение с помощью метода Гаусса с выбором ведущего элемента:")
    print(gaussian_solution)

    # Решение с использованием LU-разложения
    lu_solution = solve_linear_system(A, F, 'LU')
    print("Решение с помощью LU-разложения:")
    print(lu_solution)

    # Решение с использованием метода Якоби
    jacobi_solution = solve_linear_system(A, F, 'Jacobi')
    print("Решение с помощью метода Якоби:")
    print(jacobi_solution)

    # Вычисление ошибок и остатков
    gaussian_error = np.linalg.norm(exact_solution - gaussian_solution)
    lu_error = np.linalg.norm(exact_solution - lu_solution)
    jacobi_error = np.linalg.norm(exact_solution - jacobi_solution)

    gaussian_residual = np.linalg.norm(np.dot(A, gaussian_solution) - F)
    lu_residual = np.linalg.norm(np.dot(A, lu_solution) - F)
    jacobi_residual = np.linalg.norm(np.dot(A, jacobi_solution) - F)

    print("Ошибки:")
    print(f"Метод Гаусса с выбором ведущего элемента: {gaussian_error}")
    print(f"LU-разложение: {lu_error}")
    print(f"Метод Якоби: {jacobi_error}")

    print("Остатки:")
    print(f"Метод Гаусса с выбором ведущего элемента: {gaussian_residual}")
    print(f"LU-разложение: {lu_residual}")
    print(f"Метод Якоби: {jacobi_residual}")

    print("-----------------------------")


Исследование для k=1
Точное решение:
[ 10.66666667 -17.5         16.83333333]
Решение с помощью метода Гаусса с выбором ведущего элемента:
[  9.44444444 -15.66666667  15.        ]
Решение с помощью LU-разложения:
[  9.44444444 -15.66666667  15.        ]
Решение с помощью метода Якоби:
[  9 -15  15]
Ошибки:
Метод Гаусса с выбором ведущего элемента: 2.866365186558762
LU-разложение: 2.866365186558762
Метод Якоби: 3.5197853469904783
Остатки:
Метод Гаусса с выбором ведущего элемента: 1.7763568394002505e-15
LU-разложение: 1.7763568394002505e-15
Метод Якоби: 2.0
-----------------------------
Исследование для k=2
Точное решение:
[-16.66666667 151.         -34.33333333]
Решение с помощью метода Гаусса с выбором ведущего элемента:
[-16.33333333 147.         -33.33333333]
Решение с помощью LU-разложения:
[-16.33333333 147.         -33.33333333]
Решение с помощью метода Якоби:
[-16 145 -33]
Ошибки:
Метод Гаусса с выбором ведущего элемента: 4.1365578819970095
LU-разложение: 4.1365578819970095
Метод

###5. Оценить зависимость числа обусловленности и точности полученного решения в зависимости от параметра k.

In [ ]:
import numpy as np

def create_matrix(k, n):
    A = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if i == 0:
                A[i, j] = -1 * A[i+1, j] + 10**(-k)
            else:
                A[i, j] = np.random.choice([-4, -3, -2, -1, 0])

    return A

def create_vector(n):
    return np.arange(1, n+1)

def compute_condition_number(A):
    return np.linalg.cond(A)

def compute_solution_error(exact_solution, approximate_solution):
    return np.linalg.norm(exact_solution - approximate_solution)

# Параметры исследования
k_values = [1, 2, 3, 4, 5]
n = 3

# Исследование для каждого значения k
for k in k_values:
    print(f"Исследование для k={k}")
    A = create_matrix(k, n)
    F = create_vector(n)

    # Точное решение
    exact_solution = np.linalg.solve(A, F)

    # Решение с использованием метода Гаусса с выбором ведущего элемента
    gaussian_solution = gaussian_elimination(A, F)

    # Решение с использованием LU-разложения
    try:
        L, U = lu_decomposition(A)
        y = np.linalg.solve(L, F)
        lu_solution = np.linalg.solve(U, y)
    except np.linalg.LinAlgError:
        lu_solution = gaussian_solution

    # Решение с использованием метода Якоби
    x0 = np.zeros_like(F)
    jacobi_solution, _, _ = jacobi_method(A, F, x0)

    # Вычисление числа обусловленности
    condition_number = compute_condition_number(A)

    # Вычисление ошибок
    gaussian_error = compute_solution_error(exact_solution, gaussian_solution)
    lu_error = compute_solution_error(exact_solution, lu_solution)
    jacobi_error = compute_solution_error(exact_solution, jacobi_solution)

    print("Число обусловленности:", condition_number)
    print("Ошибки:")
    print(f"Метод Гаусса с выбором ведущего элемента: {gaussian_error}")
    print(f"LU-разложение: {lu_error}")
    print(f"Метод Якоби: {jacobi_error}")

    print("-----------------------------")


Исследование для k=1
Число обусловленности: 373.9996850356385
Ошибки:
Метод Гаусса с выбором ведущего элемента: 4.534589286804254
LU-разложение: 4.534589286804254
Метод Якоби: 6.5064070986476965
-----------------------------
Исследование для k=2
Число обусловленности: 4471.3465957473345
Ошибки:
Метод Гаусса с выбором ведущего элемента: 9.532271373122546
LU-разложение: 9.532271373122546
Метод Якоби: 9.89949493661127
-----------------------------
Исследование для k=3
Число обусловленности: 30734.29269601741
Ошибки:
Метод Гаусса с выбором ведущего элемента: 7.180703308172536
LU-разложение: 7.180703308172536
Метод Якоби: 7.3484692283495345
-----------------------------
Исследование для k=4
Число обусловленности: 276042.47686728684
Ошибки:
Метод Гаусса с выбором ведущего элемента: 2.9059326290269767
LU-разложение: 2.9059326290269767
Метод Якоби: 3.24037034920393
-----------------------------
Исследование для k=5
Число обусловленности: 716160.1234526087
Ошибки:
Метод Гаусса с выбором ведущег

Из предоставленных данных видно, что с увеличением параметра k, который отвечает за диагональное преобладание матрицы, числа обусловленности матриц A^(k) также увеличиваются. Это означает, что матрицы становятся более плохо обусловленными, и решение системы линейных уравнений становится более чувствительным к погрешностям в данных или округлении.

Как следствие, ошибка решения системы уравнений с помощью метода Гаусса и LU-разложения также увеличивается с увеличением числа обусловленности. Это отражено в значениях ошибок, которые растут с каждым шагом k.

Однако метод Якоби демонстрирует более стабильную ошибку в решении, независимо от значения k. Это может быть связано с итерационным характером метода Якоби, который позволяет достигать определенной точности независимо от числа обусловленности.

Таким образом, можно сделать вывод, что чем выше числа обусловленности матрицы, тем более неустойчивыми и неточными становятся решения системы линейных уравнений при использовании прямых методов, таких как метод Гаусса и LU-разложение. В то же время, метод Якоби остается относительно стабильным при различных значениях числа обусловленности.

###6. Провести аналогичные исследования на матрицах Гильберта, которые строятся согласно формуле

In [ ]:
import numpy as np

def create_hilbert_matrix(n):
    A = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            A[i, j] = 1 / (i + j + 1)
    return A

def create_vector(n):
    return np.arange(1, n+1)

def compute_condition_number(A):
    return np.linalg.cond(A)

def compute_solution_error(exact_solution, approximate_solution):
    return np.linalg.norm(exact_solution - approximate_solution)

# Параметры исследования
n_values = [2, 3, 4, 5]

# Исследование для каждого значения n
for n in n_values:
    print(f"Исследование для n={n}")
    A = create_hilbert_matrix(n)
    F = create_vector(n)

    # Точное решение
    exact_solution = np.linalg.solve(A, F)

    # Решение с использованием метода Гаусса с выбором ведущего элемента
    gaussian_solution = gaussian_elimination(A, F)

    # Решение с использованием LU-разложения
    try:
        L, U = lu_decomposition(A)
        y = np.linalg.solve(L, F)
        lu_solution = np.linalg.solve(U, y)
    except np.linalg.LinAlgError:
        lu_solution = gaussian_solution

    # Решение с использованием метода Якоби
    x0 = np.zeros_like(F)
    jacobi_solution, _, _ = jacobi_method(A, F, x0)

    # Вычисление числа обусловленности
    condition_number = compute_condition_number(A)

    # Вычисление ошибок
    gaussian_error = compute_solution_error(exact_solution, gaussian_solution)
    lu_error = compute_solution_error(exact_solution, lu_solution)
    jacobi_error = compute_solution_error(exact_solution, jacobi_solution)

    print("Число обусловленности:", condition_number)
    print("Ошибки:")
    print(f"Метод Гаусса с выбором ведущего элемента: {gaussian_error}")
    print(f"LU-разложение: {lu_error}")
    print(f"Метод Якоби: {jacobi_error}")

    print("-----------------------------")


Исследование для n=2
Число обусловленности: 15.016740988110644
Ошибки:
Метод Гаусса с выбором ведущего элемента: 6.708203932499369
LU-разложение: 6.708203932499369
Метод Якоби: 6.708203932499373
-----------------------------
Исследование для n=3
Число обусловленности: 310.52999339176927
Ошибки:
Метод Гаусса с выбором ведущего элемента: 303.6445290137804
LU-разложение: 303.6445290137804
Метод Якоби: 302.8101715596762
-----------------------------
Исследование для n=4
Число обусловленности: 6367.883356101975
Ошибки:
Метод Гаусса с выбором ведущего элемента: 2106.367726679501
LU-разложение: 2106.367726679501
Метод Якоби: 2103.5030306607737
-----------------------------
Исследование для n=5
Число обусловленности: 283074.71255846124
Ошибки:
Метод Гаусса с выбором ведущего элемента: 36033.20457599344
LU-разложение: 36033.20457599348
Метод Якоби: 36031.143667879966
-----------------------------


Из представленных данных видно, что числа обусловленности матриц Гильберта быстро растут с увеличением размерности матрицы n. Это указывает на то, что матрицы Гильберта являются плохо обусловленными и приводят к большим ошибкам при решении систем линейных уравнений.

Метод Гаусса и LU-разложение показывают сходные значения ошибок, которые также возрастают с увеличением размерности матрицы. Это говорит о том, что оба метода чувствительны к высокому числу обусловленности и дают сопоставимую точность решения системы уравнений.

Метод Якоби также демонстрирует увеличение ошибки с увеличением размерности матрицы, но ошибка остается сопоставимой с ошибками методов Гаусса и LU-разложения. Это может быть связано с тем, что метод Якоби имеет итерационный характер и может достичь определенной точности, несмотря на высокое число обусловленности матрицы.

Таким образом, матрицы Гильберта являются плохо обусловленными, что ведет к большим ошибкам при решении систем линейных уравнений. Метод Якоби остается относительно стабильным при различных размерностях матрицы, но также не избавлен от ошибок.

###7. Сравнить между собой прямые и итерационные методы по эффективности методов в зависимости от размеров n матрицы:

In [ ]:
import time
import numpy as np

def create_matrix(n):
    return np.random.rand(n, n)

def create_vector(n):
    return np.random.rand(n)

def measure_execution_time(method, *args):
    start_time = time.time()
    method(*args)
    end_time = time.time()
    execution_time = end_time - start_time
    return execution_time

# Размеры матрицы
n_values = [10, 50, 10**2]

# Максимальное количество итераций для итерационного метода
max_iterations = 1000

# Измерение времени выполнения для каждого значения n
for n in n_values:
    print(f"Измерение для n={n}")

    A = create_matrix(n)
    b = create_vector(n)
    x0 = np.zeros_like(b)

    # Измерение времени выполнения метода Гаусса с выбором ведущего элемента
    gaussian_time = measure_execution_time(gaussian_elimination, A, b)

    # Измерение времени выполнения LU-разложения
    lu_time = measure_execution_time(lu_decomposition, A)

    # Измерение времени выполнения метода Якоби
    jacobi_time = measure_execution_time(jacobi_method, A, b, x0, max_iterations)

    print("Время выполнения:")
    print(f"Метод Гаусса с выбором ведущего элемента: {gaussian_time} сек")
    print(f"LU-разложение: {lu_time} сек")
    print(f"Метод Якоби: {jacobi_time} сек")

    print("-----------------------------")


Измерение для n=10
Время выполнения:
Метод Гаусса с выбором ведущего элемента: 0.0007269382476806641 сек
LU-разложение: 0.0002968311309814453 сек
Метод Якоби: 0.0011372566223144531 сек
-----------------------------
Измерение для n=50
Время выполнения:
Метод Гаусса с выбором ведущего элемента: 0.006933927536010742 сек
LU-разложение: 0.0006587505340576172 сек
Метод Якоби: 0.028672456741333008 сек
-----------------------------
Измерение для n=100
Время выполнения:
Метод Гаусса с выбором ведущего элемента: 0.0263674259185791 сек
LU-разложение: 0.0031461715698242188 сек
Метод Якоби: 0.5692586898803711 сек
-----------------------------


Из полученных результатов видно, что время выполнения прямых методов (Гаусса, LU-разложения) растет значительно медленее, чем время выполнения итерационного метода (Якоби), с увеличением размерности матрицы.